<a href="https://colab.research.google.com/github/AURORARISE/MSSP-6070/blob/main/week9_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import requests

def build_flickr_url(query: str) -> str:
    """
Construct the URL of the Flickr public image feed based on the query keywords.
Here, the tags parameter is used to search for photos with specific tags.
    """
    base_url = "https://www.flickr.com/services/feeds/photos_public.gne"
    return f"{base_url}?format=json&nojsoncallback=1&tags={query}"

def get_feed_json(url: str) -> dict:
    """Download the JSON of the Flickr feed."""
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0 Safari/537.36"
        )
    }
    print(f"[INFO] Fetching Flickr feed: {url}")
    res = requests.get(url, headers=headers, timeout=15)
    res.raise_for_status()
    return res.json()

def extract_image_urls(feed_json: dict):
    """
Extract image URLs from Flickr's feed JSON.
Each item has a media field containing an'm 'key, which is a medium sized image link.
    """
    items = feed_json.get("items", [])
    image_urls = []

    for item in items:
        media = item.get("media", {})
        img_url = media.get("m")
        if img_url:
            image_urls.append(img_url)

    return image_urls

def download_image(url: str, folder: str, index: int):
    """Download a single image and save it to the specified folder."""
    try:
        print(f"[INFO] Downloading image {index}: {url}")
        res = requests.get(url, timeout=20)
        res.raise_for_status()

        filename = os.path.basename(url.split("?")[0])
        if not filename:
            filename = f"image_{index}.jpg"

        file_path = os.path.join(folder, filename)

        with open(file_path, "wb") as f:
            f.write(res.content)

        print(f"[OK] Saved to {file_path}")
    except Exception as e:
        print(f"[ERROR] Failed to download {url}: {e}")

def main():
    print("=== Flickr Image Downloader ===")
    query = input("Enter a search term (e.g., cats, sunset, cars): ").strip()

    if not query:
        print("No search term provided. Exiting.")
        return

    folder_name = f"flickr_{query.replace(' ', '_')}"
    os.makedirs(folder_name, exist_ok=True)
    print(f"[INFO] Images will be saved to folder: {folder_name}")

    feed_url = build_flickr_url(query)

    try:
        feed_json = get_feed_json(feed_url)
    except Exception as e:
        print(f"[ERROR] Failed to fetch Flickr feed: {e}")
        return

    image_urls = extract_image_urls(feed_json)
    print(f"[INFO] Found {len(image_urls)} image URLs.")

    if not image_urls:
        print("No images found. Try another keyword.")
        return

    for idx, img_url in enumerate(image_urls, start=1):
        download_image(img_url, folder_name, idx)

    print("[DONE] All possible images have been processed.")

if __name__ == "__main__":
    main()

=== Flickr Image Downloader ===
Enter a search term (e.g., cats, sunset, cars): cats
[INFO] Images will be saved to folder: flickr_cats
[INFO] Fetching Flickr feed: https://www.flickr.com/services/feeds/photos_public.gne?format=json&nojsoncallback=1&tags=cats
[INFO] Found 20 image URLs.
[INFO] Downloading image 1: https://live.staticflickr.com/65535/54919206337_2deae329ba_m.jpg
[OK] Saved to flickr_cats/54919206337_2deae329ba_m.jpg
[INFO] Downloading image 2: https://live.staticflickr.com/65535/54919703361_153f337b55_m.jpg
[OK] Saved to flickr_cats/54919703361_153f337b55_m.jpg
[INFO] Downloading image 3: https://live.staticflickr.com/65535/54918757362_4815f084a5_m.jpg
[OK] Saved to flickr_cats/54918757362_4815f084a5_m.jpg
[INFO] Downloading image 4: https://live.staticflickr.com/65535/54919802915_d66461b3bb_m.jpg
[OK] Saved to flickr_cats/54919802915_d66461b3bb_m.jpg
[INFO] Downloading image 5: https://live.staticflickr.com/65535/54918914216_07b1d64972_m.jpg
[OK] Saved to flickr_cats/5